# RAG文本切分

在大模型应用中，时常会出现由于RAG知识库参考内容过多或者长期记忆的内容过长，导致输入模型的成本过高或者超出模型的承载长度。该怎么办呢？

提高语言模型应用程序性能的最有效策略之一是将大数据分成较小的部分。后面的一系列分享将介绍文本切分的一些概念、方法和实战；会涉及很多内容，但如果你能坚持到最后，保证你会对分块理论有一个扎实的掌握。

## 简单的字符长度切分 Character Splitting 

字符分割是分割文本的最基本形式。它是简单地将文本分成n个字符大小的块的过程，而不考虑其内容或形式。

不建议在任何应用程序中使用这种方法，但它是我们理解基础知识的一个很好的起点。

* 优点:简单容易

* 缺点:非常死板，没有考虑到文本的结构

两个小概念:

**Chunk Size(块大小)**-块中包含的字符数量。50、100、10万等等。

**Chunk Overlap(块重叠)**-为了避免重要信息被切开成多个部分，希望有连续块重叠，overlap就是重叠块的大小。

In [1]:
text = "在大模型应用中，时常会出现由于知识库参考内容过多或者长期记忆的内容过长，导致输入模型的成本过高或者超出模型的承载长度。该怎么办呢？提高语言模型应用程序性能的最有效策略之一是将大数据分成较小的部分。后面的一系列分享将介绍文本切分的一些概念、方法和实战；会涉及很多内容，但如果你能坚持到最后，保证你会对分块理论有一个扎实的掌握。"

chunks = []
chunk_size = 35

for i in range(0, len(text), chunk_size):
    chunk = text[i:i + chunk_size]  # 每隔35是一个块
    chunks.append(chunk)
print(chunks)

['在大模型应用中，时常会出现由于知识库参考内容过多或者长期记忆的内容过长', '，导致输入模型的成本过高或者超出模型的承载长度。该怎么办呢？提高语言模', '型应用程序性能的最有效策略之一是将大数据分成较小的部分。后面的一系列分', '享将介绍文本切分的一些概念、方法和实战；会涉及很多内容，但如果你能坚持', '到最后，保证你会对分块理论有一个扎实的掌握。']


### LangChain实现

In [3]:
from langchain.text_splitter import CharacterTextSplitter

text = "在大模型应用中，时常会出现由于知识库参考内容过多或者长期记忆的内容过长，导致输入模型的成本过高或者超出模型的承载长度。该怎么办呢？提高语言模型应用程序性能的最有效策略之一是将大数据分成较小的部分。后面的一系列分享将介绍文本切分的一些概念、方法和实战；会涉及很多内容，但如果你能坚持到最后，保证你会对分块理论有一个扎实的掌握。"

#### 切分文档
text内容按35个字符被切分成多个片段

In [4]:
text_splitter = CharacterTextSplitter(
    chunk_size=35, 
    chunk_overlap=0, 
    separator='', 
    strip_whitespace=False
)
print(text_splitter.split_text(text))

['在大模型应用中，时常会出现由于知识库参考内容过多或者长期记忆的内容过长', '，导致输入模型的成本过高或者超出模型的承载长度。该怎么办呢？提高语言模', '型应用程序性能的最有效策略之一是将大数据分成较小的部分。后面的一系列分', '享将介绍文本切分的一些概念、方法和实战；会涉及很多内容，但如果你能坚持', '到最后，保证你会对分块理论有一个扎实的掌握。']


#### 加入overlap
通过加入overlap，我们可以发现下面的：的内容过长 在两个片段中出现了。

In [7]:
text_splitter = CharacterTextSplitter(
    chunk_size=35, 
    chunk_overlap=5, 
    separator='', 
    strip_whitespace=False
)
st = text_splitter.split_text(text)
st

['在大模型应用中，时常会出现由于知识库参考内容过多或者长期记忆的内容过长',
 '的内容过长，导致输入模型的成本过高或者超出模型的承载长度。该怎么办呢？',
 '怎么办呢？提高语言模型应用程序性能的最有效策略之一是将大数据分成较小的',
 '分成较小的部分。后面的一系列分享将介绍文本切分的一些概念、方法和实战；',
 '法和实战；会涉及很多内容，但如果你能坚持到最后，保证你会对分块理论有一',
 '块理论有一个扎实的掌握。']

#### 加入separator

利用separator将文档切分成小块，再使用长度对小块进行切割；

strip_whitespace：指定是否在分割文本时删除空白字符

In [9]:
text_splitter = CharacterTextSplitter(
    chunk_size=35, 
    chunk_overlap=0, 
    separator='，', 
    strip_whitespace=False
)
st = text_splitter.split_text(text)
st

Created a chunk of size 96, which is longer than the specified 35


['在大模型应用中，时常会出现由于知识库参考内容过多或者长期记忆的内容过长',
 '导致输入模型的成本过高或者超出模型的承载长度。该怎么办呢？提高语言模型应用程序性能的最有效策略之一是将大数据分成较小的部分。后面的一系列分享将介绍文本切分的一些概念、方法和实战；会涉及很多内容',
 '但如果你能坚持到最后，保证你会对分块理论有一个扎实的掌握。']

## 通过分隔符切分，然后递归合并 Recursive Character Text Splitting

按字符长度切分的问题在于，根本没有考虑到文档的结构。只是按固定数量的字符分割，这样很容易造成文档被随意切分开了，导致最后的效果不好。
递归字符文本分割器可以帮助解决这个问题。有了它，切分的过程可描述为：我们先指定一系列分隔符，这些分隔符用于分割文档，然后将分割好的小块合并，达到指定的长度。

它是切分器中最基础的做法，也是快速搭建应用流程的首选。如果在不知道选择哪个切分器的时候，这是一个不错的选择。

### LangChain实现

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:

text = """
One of the most important things I didn't understand about the world when I was a child is the degree to which the returns for performance are superlinear.

Teachers and coaches implicitly told us the returns were linear. "You get out," I heard a thousand times, "what you put in." They meant well, but this is rarely true. If your product is only half as good as your competitor's, you don't get half as many customers. You get no customers, and you go out of business.

It's obviously true that the returns for performance are superlinear in business. Some think this is a flaw of capitalism, and that if we changed the rules it would stop being true. But superlinear returns for performance are a feature of the world, not an artifact of rules we've invented. We see the same pattern in fame, power, military victories, knowledge, and even benefit to humanity. In all of these, the rich get richer. [1]
"""

text_splitter = RecursiveCharacterTextSplitter(chunk_size=65, chunk_overlap=0)
docs = text_splitter.create_documents([text])
docs

[Document(page_content="One of the most important things I didn't understand about the"),
 Document(page_content='world when I was a child is the degree to which the returns for'),
 Document(page_content='performance are superlinear.'),
 Document(page_content='Teachers and coaches implicitly told us the returns were linear.'),
 Document(page_content='"You get out," I heard a thousand times, "what you put in." They'),
 Document(page_content='meant well, but this is rarely true. If your product is only'),
 Document(page_content="half as good as your competitor's, you don't get half as many"),
 Document(page_content='customers. You get no customers, and you go out of business.'),
 Document(page_content="It's obviously true that the returns for performance are"),
 Document(page_content='superlinear in business. Some think this is a flaw of'),
 Document(page_content='capitalism, and that if we changed the rules it would stop being'),
 Document(page_content='true. But superlinear returns for

拆分器首先寻找\n\n(段落断行)，一旦段落被分割，它就会查看块的大小，如果一个块太大，它就会被下一个分隔符分割。如果该块仍然太大，那么它将移动到下一个，依此类推。

## 切分时融入token切分
语言模型，不管是大语言模型，还是向量模型、重排模型等等，都是有长度限制的，我们在切分文档时使用token的个数进行切分，有以下好处：
- 保证切分后的文本片段不超过模型最大长度；
- 保证文本片段送入模型的token数相当，并行节约资源；
- 可以有效预估送入大模型的片段长度，便于系统内资源调整。

我们可以很容易的通过tiktoken在切分器中融入token切分工具

In [13]:
text = """8月12日，中国乒乓球界的一则趣闻在网络上持续发酵，引发了广泛关注和热议。在巴黎奥运会期间的一次采访环节中，中国乒乓球队的绝对主力马龙在接受媒体采访时，其队友王楚钦与樊振东在镜头后的轻松“划水”画面被捕捉并上传至网络，迅速走红。这一幕不仅展现了国乒队员之间轻松愉快的氛围，也意外成为了公众茶余饭后的谈资。

面对这一突如其来的“网红”事件，马龙在随后的一次做客节目中，以他特有的幽默与谦逊方式进行了回应。马龙笑称：“看来今后王楚钦、樊振东的摸鱼机会不多了，其实他们两位的表达能力远胜于我，只是平时在训练场上太过勤快，以至于在这方面偶尔犯起了‘懒’。”"""

In [15]:

from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", 
    chunk_size=50, 
    chunk_overlap=0
)
texts = text_splitter.split_text(text)
texts

Created a chunk of size 201, which is longer than the specified 50


['8月12日，中国乒乓球界的一则趣闻在网络上持续发酵，引发了广泛关注和热议。在巴黎奥运会期间的一次采访环节中，中国乒乓球队的绝对主力马龙在接受媒体采访时，其队友王楚钦与樊振东在镜头后的轻松“划水”画面被捕捉并上传至网络，迅速走红。这一幕不仅展现了国乒队员之间轻松愉快的氛围，也意外成为了公众茶余饭后的谈资。',
 '面对这一突如其来的“网红”事件，马龙在随后的一次做客节目中，以他特有的幽默与谦逊方式进行了回应。马龙笑称：“看来今后王楚钦、樊振东的摸鱼机会不多了，其实他们两位的表达能力远胜于我，只是平时在训练场上太过勤快，以至于在这方面偶尔犯起了‘懒’。”']

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4",
    chunk_size=50,
    chunk_overlap=0,
)
texts = text_splitter.split_text(text)
texts

['8月12日，中国乒乓球界的一则趣闻在网络上持续发酵，引发了广泛关注和热议。在巴',
 '黎奥运会期间的一次采访环节中，中国乒乓球队的绝对主力马龙在接受媒体采访时',
 '，其队友王楚钦与樊振东在镜头后的轻松“划水”画面被捕捉并上传至网络，迅速',
 '走红。这一幕不仅展现了国乒队员之间轻松愉快的氛围，也意外成为了公众茶余',
 '饭后的谈资。',
 '面对这一突如其来的“网红”事件，马龙在随后的一次做客节目中，以他特有的幽默与谦',
 '逊方式进行了回应。马龙笑称：“看来今后王楚钦、樊振东的摸鱼机会不多了，其实他',
 '们两位的表达能力远胜于我，只是平时在训练场上太过勤快，以至于在这方面偶尔犯起了‘',
 '懒’。”']

## 针对不同文档格式切分 Document Specific Splitting
在日常数据处理中，不仅仅有txt数据，还包含一些存在结构的数据，例如json、markdown、代码(例如py)、PDF等。

让我们看看如何处理JSON数据

首先遍历json数据深度，然后构建更小的json块。试图保持嵌套json对象的完整，但在需要的时候会将它们分开，以保持块的大小在min_chunk_size和max_chunk_size之间。

如果值不是一个嵌套的json，而是一个非常大的字符串，那么字符串将不会被分割。如果您需要对块大小进行硬性限制，请考虑在这些块上使用递归文本分配器来组合它。拆分列表有一个可选的预处理步骤

首先将它们转换为json (dict)，然后拆分它们。

In [17]:
import requests

# This is a large nested json object and will be loaded as a python dict
json_data = requests.get("https://api.smith.langchain.com/openapi.json").json()
json_data

{'openapi': '3.1.0',
 'info': {'title': 'LangSmith', 'version': '0.1.0'},
 'paths': {'/api/v1/sessions/{session_id}': {'get': {'tags': ['tracer-sessions'],
    'summary': 'Read Tracer Session',
    'description': 'Get a specific session.',
    'operationId': 'read_tracer_session_api_v1_sessions__session_id__get',
    'security': [{'API Key': []}, {'Tenant ID': []}, {'Bearer Auth': []}],
    'parameters': [{'name': 'session_id',
      'in': 'path',
      'required': True,
      'schema': {'type': 'string', 'format': 'uuid', 'title': 'Session Id'}},
     {'name': 'include_stats',
      'in': 'query',
      'required': False,
      'schema': {'type': 'boolean',
       'default': False,
       'title': 'Include Stats'}},
     {'name': 'accept',
      'in': 'header',
      'required': False,
      'schema': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
       'title': 'Accept'}}],
    'responses': {'200': {'description': 'Successful Response',
      'content': {'application/json': {'sch

定义切分器;切分文档

In [19]:

from langchain_text_splitters import RecursiveJsonSplitter

splitter = RecursiveJsonSplitter(max_chunk_size=300)
json_chunks = splitter.split_json(json_data=json_data)
json_chunks

[{'openapi': '3.1.0',
  'info': {'title': 'LangSmith', 'version': '0.1.0'},
  'paths': {'/api/v1/sessions/{session_id}': {'get': {'tags': ['tracer-sessions'],
     'summary': 'Read Tracer Session',
     'description': 'Get a specific session.'}}}},
 {'paths': {'/api/v1/sessions/{session_id}': {'get': {'operationId': 'read_tracer_session_api_v1_sessions__session_id__get',
     'security': [{'API Key': []}, {'Tenant ID': []}, {'Bearer Auth': []}]}}}},
 {'paths': {'/api/v1/sessions/{session_id}': {'get': {'parameters': [{'name': 'session_id',
       'in': 'path',
       'required': True,
       'schema': {'type': 'string', 'format': 'uuid', 'title': 'Session Id'}},
      {'name': 'include_stats',
       'in': 'query',
       'required': False,
       'schema': {'type': 'boolean',
        'default': False,
        'title': 'Include Stats'}},
      {'name': 'accept',
       'in': 'header',
       'required': False,
       'schema': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
        '